In [ ]:
# Install tfx and kfp Python packages.
import sys
import os

!{sys.executable} -m pip install --user --upgrade -q tfx
!{sys.executable} -m pip install --user --upgrade -q kfp
# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/

In [ ]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

In [ ]:
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='https://39aee44fc104321f-dot-us-central2.pipelines.googleusercontent.com'
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/house_price_pipeline'

In [ ]:
TEMPLATE_NAME="taxi_template"
PIPELINE_NAME="house_price_pipeline"

TEMPLATE_DIR=os.path.join(os.path.expanduser("~"),"imported",TEMPLATE_NAME)
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

In [ ]:
# !tfx template copy \
#   --pipeline-name={TEMPLATE_NAME} \
#   --destination-path={TEMPLATE_DIR} \
#   --model=taxi

In [ ]:
cd {"~/imported/house_price_pipeline"}

## Add dataset to GCS bucket

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-hprice/data/data.csv

In [ ]:
!tfx pipeline create  \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}